In [1]:
from Toolbox_GCSR.Preprocessing import Processor
from sklearn.preprocessing import minmax_scale
import numpy as np
from Toolbox_GCSR.utility import eval_band, eval_band_cv
from Toolbox_GCSR.EGCSR_BS_Ranking import EGCSR_BS_Ranking
from pathlib import Path
import pandas as pd
from scipy.io import loadmat
import time


# im_, gt_ = 'Pavia', 'Pavia_gt'
# im_, gt_ = 'PaviaU', 'PaviaU_gt'
# im_, gt_ = 'Salinas_corrected', 'Salinas_gt'
# im_, gt_ = 'Botswana', 'Botswana_gt'
# im_, gt_ = 'KSC', 'KSC_gt'

In [2]:
#Test to turn corn spectral firms into squeare matrix

In [3]:
root = 'D:\Python\HSI_Files\\'
im_, gt_ = 'CornData', 'CornLabels'

'''
CornData = loadmat('CornData.mat')['Corn']
CornData = CornData.T
CornLabelPath = Path(r"C:\Datasets\Corn2021\15-septiembre-2021\Identificador.txt")
CornLabels = pd.read_csv(CornLabelPath, sep = ',', header = [0])
'''

'\nCornData = loadmat(\'CornData.mat\')[\'Corn\']\nCornData = CornData.T\nCornLabelPath = Path(r"C:\\Datasets\\Corn2021\r-septiembre-2021\\Identificador.txt")\nCornLabels = pd.read_csv(CornLabelPath, sep = \',\', header = [0])\n'

In [4]:
img_path = root + im_ + '.mat'
gt_path = root + gt_ + '.mat'
print(img_path)

p = Processor()
img, gt = p.prepare_data(img_path, gt_path)

print(type(img), type(gt))
print(img.shape, gt.shape)

n_column, n_band = img.shape
X_img = minmax_scale(img.reshape(n_column, n_band).transpose())
X_img = X_img.transpose().reshape((n_column, n_band))
img_correct, gt_correct = p.get_correct2D(X_img, gt)
gt_correct = p.standardize_label(gt_correct)
X_img_2D = X_img.reshape(n_column, n_band)
X_img_2D = minmax_scale(X_img_2D.transpose()).transpose()


D:\Python\HSI_Files\CornData.mat
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(447, 1251) (447, 1)


In [5]:
n_selected_band = 5

algorithm = [
                #EGCSR_BS_Clustering(n_selected_band, regu_coef=1e4, n_neighbors=3, ro=0.8),
                EGCSR_BS_Ranking(n_selected_band, regu_coef=1e4, n_neighbors=3, ro=0.8)
                ]

alg_key = ['EGCSR-Clustering', 'EGCSR-Ranking']
for i in range(algorithm.__len__()):
    time_start = time.process_time()
    X_new = np.array(algorithm[i].predict(X_img_2D), dtype="object")
    run_time = round(time.process_time() - time_start)
    # if X_new.shape[0] < n_selected_band:

    #Creates new image with selected bands
    #TODO: Fix IndexError: tuple index out of range
    #X_new_3D = X_new.reshape((n_row, n_column, X_new.shape[1]))
    
    gt_correct = p.standardize_label(gt_correct)
    score = eval_band_cv(img_correct, gt_correct, times=10)
    print('%s knn:  %.4f + %.4f' % (alg_key[i], score['knn']['oa'][0], score['knn']['oa'][1]))
    print('%s svm:  %.4f + %.4f' % (alg_key[i], score['svm']['oa'][0], score['svm']['oa'][1]))
    print('-------------------------------------------')


band index: [1045 1048 1047 1046 1248]
band indexlength: 5
EGCSR-Clustering knn:  41.6900 + 1.8800
EGCSR-Clustering svm:  52.3600 + 1.4500
-------------------------------------------


In [13]:
#Selecting bands with Ranking algorithm
iterations = 2
factor = 5

bands_GCSR = np.zeros((iterations,iterations * factor))

for i in range(4):
    alg_Ranking = EGCSR_BS_Ranking((i+1)*factor, regu_coef=1e2, n_neighbors=3, ro=0.8)
    selected_img = np.array(alg_Ranking.predict(X_img_2D), dtype=object)
    selected_bands = alg_Ranking.band_indx
    print("image shape: ", selected_img.shape)
    print("desired bands",(i+1)*factor, "Total bands" , selected_bands.shape,"\n")
    b, = selected_bands.shape
    print(selected_bands,"\n")
    bands_GCSR[i,:b] = selected_bands



band index: [1203 1199 1200 1201 1202]
band indexlength: 5
image shape:  (447, 5)
desired bands 5 Total bands (5,) 

[1203 1199 1200 1201 1202] 

band index: [1138  427 1198 1137  399 1203 1199 1200 1201 1202]
band indexlength: 10
image shape:  (447, 10)
desired bands 10 Total bands (10,) 

[1138  427 1198 1137  399 1203 1199 1200 1201 1202] 

band index: [ 219  221  401  428  400 1138  427 1198 1137  399 1203 1199 1200 1201
 1202]
band indexlength: 15
image shape:  (447, 15)
desired bands 15 Total bands (15,) 

[ 219  221  401  428  400 1138  427 1198 1137  399 1203 1199 1200 1201
 1202] 



IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
#saves selected bands
np.save("bands_GCSR", bands_GCSR)